In [2]:
import sys
sys.path.append('../')

In [3]:
from scripts.utils import data_path, set_mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import h5py


%matplotlib inline

set_mpl()

matplotlib settings set
matplotlib settings set


In [6]:
data_std = pd.read_pickle(data_path+'London/london_std.pkl_gz', compression='gzip')
data_std = data_std.resample('1H').mean()
data_std.head(10)

id,MAC000002,MAC000003,MAC000004,MAC000006,MAC000007,MAC000008,MAC000009,MAC000010,MAC000011,MAC000012,MAC000013,MAC000016,MAC000018,MAC000019,MAC000020,MAC000021,MAC000022,MAC000023,MAC000024,MAC000025,MAC000026,MAC000027,MAC000028,MAC000029,MAC000030,MAC000032,MAC000033,MAC000034,MAC000035,MAC000036,MAC000037,MAC000038,MAC000039,MAC000040,MAC000041,MAC000042,MAC000043,MAC000045,MAC000047,MAC000049,MAC000050,MAC000053,MAC000054,MAC000055,MAC000056,MAC000057,MAC000058,MAC000059,MAC000060,MAC000061,MAC000062,MAC000063,MAC000066,MAC000067,MAC000068,MAC000069,MAC000070,MAC000072,MAC000073,MAC000074,MAC000077,MAC000078,MAC000079,MAC000080,MAC000081,MAC000083,MAC000084,MAC000085,MAC000086,MAC000087,MAC000089,MAC000090,MAC000091,MAC000092,MAC000093,MAC000094,MAC000095,MAC000096,MAC000097,MAC000098,MAC000099,MAC000100,MAC000101,MAC000102,MAC000103,MAC000104,MAC000105,MAC000108,MAC000110,MAC000111,MAC000112,MAC000113,MAC000114,MAC000115,MAC000116,MAC000117,MAC000118,MAC000119,MAC000120,MAC000121,MAC000122,MAC000123,MAC000125,MAC000126,MAC000127,MAC000128,MAC000129,MAC000130,MAC000131,MAC000132,MAC000133,MAC000134,MAC000135,MAC000136,MAC000137,MAC000138,MAC000140,MAC000142,MAC000143,MAC000144,MAC000145,MAC000148,MAC000149,MAC000150,MAC000151,MAC000152,MAC000153,MAC000154,MAC000155,MAC000156,MAC000157,MAC000159,MAC000160,MAC000161,MAC000162,MAC000163,MAC000164,MAC000166,MAC000167,MAC000168,MAC000169,MAC000171,MAC000172,MAC000174,MAC000175,MAC000176,MAC000177,MAC000178,MAC000179,MAC000180,MAC000181,MAC000182,MAC000183,MAC000184,MAC000185,MAC000188,MAC000189,MAC000190,MAC000191,MAC000192,MAC000196,MAC000197,MAC000199,MAC000200,MAC000201,MAC000202,MAC000203,MAC000204,MAC000205,MAC000206,MAC000207,MAC000208,MAC000209,MAC000210,MAC000211,MAC000212,MAC000213,MAC000214,MAC000215,MAC000216,MAC000217,MAC000218,MAC000220,MAC000221,MAC000222,MAC000223,MAC000224,MAC000225,MAC000226,MAC000227,MAC000228,MAC000229,MAC000230,MAC000231,MAC000232,MAC000233,MAC000234,MAC000235,MAC000237,MAC000238,MAC000239,MAC000240,MAC000241,MAC000242,MAC000243,MAC000244,MAC000245,MAC000246,MAC000248,MAC000249,MAC000250,MAC000251,MAC000252,MAC000253,MAC000254,MAC000255,MAC000256,MAC000258,MAC000260,MAC000262,MAC000263,MAC000264,MAC000267,MAC000268,MAC000269,MAC000270,MAC000271,MAC000272,MAC000273,MAC000274,MAC000275,MAC000276,MAC000277,MAC000279,MAC000280,MAC000281,MAC000282,MAC000283,MAC000284,MAC000285,MAC000287,MAC000289,MAC000291,MAC000294,MAC000295,MAC000296,MAC000297,MAC000299,MAC000300,MAC000301,...,MAC005245,MAC005246,MAC005247,MAC005248,MAC005249,MAC005251,MAC005252,MAC005253,MAC005254,MAC005257,MAC005258,MAC005259,MAC005260,MAC005261,MAC005262,MAC005263,MAC005264,MAC005265,MAC005267,MAC005268,MAC005269,MAC005270,MAC005272,MAC005273,MAC005274,MAC005275,MAC005276,MAC005277,MAC005280,MAC005282,MAC005283,MAC005285,MAC005286,MAC005287,MAC005288,MAC005289,MAC005293,MAC005294,MAC005295,MAC005296,MAC005297,MAC005298,MAC005299,MAC005300,MAC005301,MAC005302,MAC005303,MAC005304,MAC005305,MAC005307,MAC005309,MAC005311,MAC005312,MAC005313,MAC005314,MAC005315,MAC005316,MAC005317,MAC005318,MAC005319,MAC005320,MAC005322,MAC005323,MAC005325,MAC005326,MAC005327,MAC005328,MAC005329,MAC005330,MAC005331,MAC005332,MAC005333,MAC005334,MAC005335,MAC005336,MAC005337,MAC005338,MAC005339,MAC005340,MAC005343,MAC005344,MAC005345,MAC005346,MAC005348,MAC005350,MAC005351,MAC005353,MAC005355,MAC005356,MAC005357,MAC005358,MAC005360,MAC005361,MAC005362,MAC005363,MAC005364,MAC005365,MAC005366,MAC005367,MAC005369,MAC005370,MAC005371,MAC005372,MAC005373,MAC005374,MAC005375,MAC005376,MAC005377,MAC005378,MAC005379,MAC005380,MAC005381,MAC005382,MAC005383,MAC005385,MAC005386,MAC005387,MAC005388,MAC005389,MAC005390,MAC005391,MAC005392,MAC005393,MAC005394,MAC005397,MAC005398,MAC005399,MAC005400,MAC005402,MAC005403,MAC005404,MAC005405,MAC005406,MAC005407,MAC005408,MAC005409,MAC005410,MAC005411,MAC005412,MAC005414,MAC005415,MAC005416,MAC005417,MAC005418,MAC005419,MAC005420,MAC005421,MAC005422,MAC005423,MAC